In [1]:
import numpy as np
import lmdb
import os
import pickle
import torch
from iopath.common.file_io import PathManager as pm
import json

PathManager = pm()

In [9]:
train_data_path = os.path.join("/home/zmykevin/fb_intern/data/mmf_data/datasets/flickr30k/defaults/annotations", "flickr30k_itm_train_final.jsonl")
data = []
with open(train_data_path, "r") as f:
    for line in f:
        d_point = json.loads(line)
        data.append(d_point)

In [10]:
print(data[0:10])


[{'filename': '1000092795.jpg', 'split': 'train', 'captions': ['Two young guys with shaggy hair look at their hands while hanging out in the yard.'], 'sentid': 0, 'label': 1}, {'filename': '1000092795.jpg', 'split': 'train', 'captions': ['Two young, White males are outside near many bushes.'], 'sentid': 1, 'label': 1}, {'filename': '1000092795.jpg', 'split': 'train', 'captions': ['Two men in green shirts are standing in a yard.'], 'sentid': 2, 'label': 1}, {'filename': '1000092795.jpg', 'split': 'train', 'captions': ['A man in a blue shirt standing in a garden.'], 'sentid': 3, 'label': 1}, {'filename': '1000092795.jpg', 'split': 'train', 'captions': ['Two friends enjoy time spent together.'], 'sentid': 4, 'label': 1}, {'filename': '10002456.jpg', 'split': 'train', 'captions': ['Several men in hard hats are operating a giant pulley system.'], 'sentid': 5, 'label': 1}, {'filename': '10002456.jpg', 'split': 'train', 'captions': ['Workers look down from up above on a piece of equipment.'],

In [6]:
#ann_data_path = "/fsx/zmykevin/data/mmf_data/datasets/visual_entailment/defaults/annotations/snli_ve_test.jsonl"
data_path = "/home/zmykevin/fb_intern/data/mmf_data/datasets/flickr30k/defaults/annotations"
#ann_data_path = "/fsx/zmykevin/data/mmf_data/datasets/flickr30k/defaults/annotations/dataset_flickr30k.json"
ann_data_path = os.path.join(data_path, "dataset_flickr30k.json")
with open(ann_data_path, "r") as f:
    ann_data = json.load(f)
print(type(ann_data))
print(ann_data["images"][0])    

<class 'dict'>
{'sentids': [0, 1, 2, 3, 4], 'imgid': 0, 'sentences': [{'tokens': ['two', 'young', 'guys', 'with', 'shaggy', 'hair', 'look', 'at', 'their', 'hands', 'while', 'hanging', 'out', 'in', 'the', 'yard'], 'raw': 'Two young guys with shaggy hair look at their hands while hanging out in the yard.', 'imgid': 0, 'sentid': 0}, {'tokens': ['two', 'young', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes'], 'raw': 'Two young, White males are outside near many bushes.', 'imgid': 0, 'sentid': 1}, {'tokens': ['two', 'men', 'in', 'green', 'shirts', 'are', 'standing', 'in', 'a', 'yard'], 'raw': 'Two men in green shirts are standing in a yard.', 'imgid': 0, 'sentid': 2}, {'tokens': ['a', 'man', 'in', 'a', 'blue', 'shirt', 'standing', 'in', 'a', 'garden'], 'raw': 'A man in a blue shirt standing in a garden.', 'imgid': 0, 'sentid': 3}, {'tokens': ['two', 'friends', 'enjoy', 'time', 'spent', 'together'], 'raw': 'Two friends enjoy time spent together.', 'imgid': 0, 'sentid': 4}], 'sp

In [7]:
#Logics Training with pre-generate data
import random

print(ann_data.keys())
flickr30k = {}
for x in ann_data["images"]:
    current_split = x['split']
    filename = x['filename']
    if flickr30k.get(current_split, None) is None:
        flickr30k[current_split] = []
        
    for sent in x["sentences"]:
        current_data_point = {}
        current_data_point["filename"] = filename
        current_data_point["split"] = current_split
        current_data_point["captions"] = [sent["raw"]]
        current_data_point["sentid"] = sent["sentid"]
        current_data_point["label"] = 1
            #
#         if current_split == "val":
#             #By 50% chance, we will have a neg_caption
#             if random.random() < 0.5:
#                 neg_sample = random.choice(ann_data["images"]) #Only Sample from Validation Images
#                 while neg_sample["filename"] == filename:
#                     neg_sample = random.choice(ann_data["images"])
#                 neg_caption = random.choice(neg_sample["sentences"])["raw"]
#                 current_data_point["neg_captions"] = [neg_caption]
#                 current_data_point["label"] = 0
#             else:
#                 current_data_point["neg_captions"] = []
#                 current_data_point["label"] = 1
                
        flickr30k[current_split].append(current_data_point)



dict_keys(['images', 'dataset'])


In [8]:
print(len(flickr30k["train"]))
print(len(flickr30k["val"]))
print(len(flickr30k["test"]))


flickr30k["minitest"] = []
for i,x in enumerate(flickr30k["train"]):
    flickr30k["minitest"].append(x)
#     if i == 999:
#         break

145000
5070
5000


In [8]:
#Design Fixed Val
old_val = flickr30k["val"].copy()
flickr30k["val"] = []
for x in old_val:
    current_data_point = x.copy()
    if random.random() < 0.5:
        neg_sample = random.choice(old_val) #Only Sample from Validation Images
        while neg_sample["filename"] == filename:
            neg_sample = random.choice(old_val)
        neg_caption = neg_sample["captions"][0]
        current_data_point["neg_captions"] = [neg_caption]
        current_data_point["label"] = 0
    else:
        current_data_point["neg_captions"] = []
    flickr30k["val"].append(current_data_point)

In [9]:
print(flickr30k["val"][:10])

[{'filename': '1018148011.jpg', 'split': 'val', 'captions': ['A group of people stand in the back of a truck filled with cotton.'], 'sentid': 335, 'label': 1, 'neg_captions': []}, {'filename': '1018148011.jpg', 'split': 'val', 'captions': ['Men are standing on and about a truck carrying a white substance.'], 'sentid': 336, 'label': 0, 'neg_captions': ['A black dog digs in the snow.']}, {'filename': '1018148011.jpg', 'split': 'val', 'captions': ['A group of people are standing on a pile of wool in a truck.'], 'sentid': 337, 'label': 1, 'neg_captions': []}, {'filename': '1018148011.jpg', 'split': 'val', 'captions': ['A group of men are loading cotton onto a truck'], 'sentid': 338, 'label': 0, 'neg_captions': ['A mounted policeman on a dappled horse surveys a crowd.']}, {'filename': '1018148011.jpg', 'split': 'val', 'captions': ['Workers load sheared wool onto a truck.'], 'sentid': 339, 'label': 0, 'neg_captions': ['A man has a pan and what looks like a block of dirt.']}, {'filename': '10

In [26]:
flickr30k["minitest_final"] = []
last_filename = ""
for j in range(0, len(flickr30k["minitest"]), 5):
    #iterate through image
    x = flickr30k["minitest"][j]
    for i,y in enumerate(flickr30k["minitest"]):
        new_x = x.copy()
        assert new_x["filename"] != last_filename
#         new_x = x.copy()
        new_x["captions"] = y["captions"].copy()
        new_x["sample_id"] = i
        new_x["label"] = int(y["filename"] == new_x["filename"])
        flickr30k["minitest_final"].append(new_x)
    last_filename = new_x["filename"]

print(len(flickr30k["minitest_final"]))
#print(flickr30k["minitest_final"][:10])

5000000


In [10]:
output_data_path = os.path.join(data_path, "flickr30k_itm_val_final.jsonl")
with open(output_data_path, 'w') as outfile:
    for entry in flickr30k["val"]:
        json.dump(entry, outfile)
        outfile.write('\n')

In [9]:
#Load the Feature
class PaddedFasterRCNNFeatureReader:
    def __init__(self, max_loc):
        self.max_loc = max_loc
        self.first = True
        self.take_item = False

    def _load(self, image_feat_path):
        image_info = {}
        image_info["features"] = load_feat(image_feat_path)

        info_path = "{}_info.npy".format(image_feat_path.split(".npy")[0])
        if PathManager.exists(info_path):
            image_info.update(load_feat(info_path).item())

        return image_info

    def read(self, image_feat_path):
        image_info = self._load(image_feat_path)
        if self.first:
            self.first = False
            if (
                image_info["features"].size == 1
                and "image_feat" in image_info["features"].item()
            ):
                self.take_item = True

        image_feature = image_info["features"]
#         print(image_info["features"].size)
        if self.take_item:
            item = image_info["features"].item()
            if "image_text" in item:
                image_info["image_text"] = item["image_text"]
                image_info["is_ocr"] = item["image_bbox_source"]
                image_feature = item["image_feat"]

            if "info" in item:
                if "image_text" in item["info"]:
                    image_info.update(item["info"])
                image_feature = item["feature"]

        # Handle case of features with class probs
        if (
            image_info["features"].size == 1
            and "features" in image_info["features"].item()
        ):
            item = image_info["features"].item()
            image_feature = item["features"]
            image_info["image_height"] = item["image_height"]
            image_info["image_width"] = item["image_width"]

            # Resize these to self.max_loc
            image_loc, _ = image_feature.shape
            image_info["cls_prob"] = np.zeros(
                (self.max_loc, item["cls_prob"].shape[1]), dtype=np.float32
            )
            image_info["cls_prob"][0:image_loc,] = item["cls_prob"][: self.max_loc, :]
            image_info["bbox"] = np.zeros(
                (self.max_loc, item["bbox"].shape[1]), dtype=np.float32
            )
            image_info["bbox"][0:image_loc,] = item["bbox"][: self.max_loc, :]
            image_info["num_boxes"] = item["num_boxes"]

        # Handle the case of ResNet152 features
        if len(image_feature.shape) > 2:
            shape = image_feature.shape
            image_feature = image_feature.reshape(-1, shape[-1])

        image_loc, image_dim = image_feature.shape
        tmp_image_feat = np.zeros((self.max_loc, image_dim), dtype=np.float32)
        tmp_image_feat[0:image_loc,] = image_feature[: self.max_loc, :]  # noqa
        image_feature = torch.from_numpy(tmp_image_feat)

        del image_info["features"]
        image_info["max_features"] = torch.tensor(image_loc, dtype=torch.long)
        return image_feature, image_info


class LMDBFeatureReader(PaddedFasterRCNNFeatureReader):
    def __init__(self, max_loc, base_path):
        super().__init__(max_loc)
        self.db_path = base_path

        if not PathManager.exists(self.db_path):
            raise RuntimeError(
                "{} path specified for LMDB features doesn't exists.".format(
                    self.db_path
                )
            )
        self.env = None

    def _init_db(self):
        self.env = lmdb.open(
            self.db_path,
            subdir=os.path.isdir(self.db_path),
            readonly=True,
            lock=False,
            readahead=False,
            meminit=False,
        )
        with self.env.begin(write=False, buffers=True) as txn:
            self.image_ids = pickle.loads(txn.get(b"keys"))
            self.image_id_indices = {
                self.image_ids[i]: i for i in range(0, len(self.image_ids))
            }

    def _load(self, image_file_path):
        #print("env is: {}".format(self.env))
        if self.env is None:
            #print("initialize db")
            self._init_db()

        split = os.path.relpath(image_file_path, self.db_path).split(".npy")[0]

        try:
            image_id = int(split.split("_")[-1])
            # Try fetching to see if it actually exists otherwise fall back to
            # default
            img_id_idx = self.image_id_indices[str(image_id).encode()]
        except (ValueError, KeyError):
            # The image id is complex or involves folder, use it directly
            image_id = str(split).encode()
            img_id_idx = self.image_id_indices[image_id]

        with self.env.begin(write=False, buffers=True) as txn:
            image_info = pickle.loads(txn.get(self.image_ids[img_id_idx]))

        return image_info

In [17]:
base_path = "/fsx/zmykevin/data/mmf_data/datasets/flickr30k/defaults/features/vinvl_detectron_sample.lmdb"

In [18]:
feature_reader = LMDBFeatureReader(100, base_path)

In [19]:
feats, info = feature_reader.read("/fsx/zmykevin/data/mmf_data/datasets/vinvl_detectron_sample.lmdb/defaults/features/detectron.lmdb/flickr30k_1000092795.npy")

In [20]:
print(feats.size())

torch.Size([100, 2048])


In [21]:
print(info.keys())

dict_keys(['image_id', 'image_height', 'image_width', 'num_boxes', 'objects', 'bbox', 'cls_prob', 'max_features'])


In [22]:
print(info['image_id'])

1000092795


In [16]:
print(type(info['image_id']))

<class 'int'>
